# Objective

This notebook is a continuation of PyTorch learning series. Following working with datasets, dataloaders, ANNs, CNNs, and RNNs for question answering, the objective of this notebook is to **build and train an LSTM model for next-word prediction**, demonstrating its application to sequential data.

In [52]:
# install nltk package
!pip install nltk

In [53]:
# import requried packages
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [54]:
# Create a document
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

## Tokenize and Build Vocabulary

In [55]:
# Convert text into lower case and create Tokens
tokens = word_tokenize(document.lower())

In [56]:
print(tokens)

['about', 'the', 'program', 'what', 'is', 'the', 'course', 'fee', 'for', 'data', 'science', 'mentorship', 'program', '(', 'dsmp', '2023', ')', 'the', 'course', 'follows', 'a', 'monthly', 'subscription', 'model', 'where', 'you', 'have', 'to', 'make', 'monthly', 'payments', 'of', 'rs', '799/month', '.', 'what', 'is', 'the', 'total', 'duration', 'of', 'the', 'course', '?', 'the', 'total', 'duration', 'of', 'the', 'course', 'is', '7', 'months', '.', 'so', 'the', 'total', 'course', 'fee', 'becomes', '799', '*', '7', '=', 'rs', '5600', '(', 'approx', '.', ')', 'what', 'is', 'the', 'syllabus', 'of', 'the', 'mentorship', 'program', '?', 'we', 'will', 'be', 'covering', 'the', 'following', 'modules', ':', 'python', 'fundamentals', 'python', 'libraries', 'for', 'data', 'science', 'data', 'analysis', 'sql', 'for', 'data', 'science', 'maths', 'for', 'machine', 'learning', 'ml', 'algorithms', 'practical', 'ml', 'mlops', 'case', 'studies', 'you', 'can', 'check', 'the', 'detailed', 'syllabus', 'here',

In [57]:
# Get unique tokens and add them into vocabulary
vocab = {'<unk>':0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

In [58]:
print(vocab)

{'<unk>': 0, 'about': 1, 'the': 2, 'program': 3, 'what': 4, 'is': 5, 'course': 6, 'fee': 7, 'for': 8, 'data': 9, 'science': 10, 'mentorship': 11, '(': 12, 'dsmp': 13, '2023': 14, ')': 15, 'follows': 16, 'a': 17, 'monthly': 18, 'subscription': 19, 'model': 20, 'where': 21, 'you': 22, 'have': 23, 'to': 24, 'make': 25, 'payments': 26, 'of': 27, 'rs': 28, '799/month': 29, '.': 30, 'total': 31, 'duration': 32, '?': 33, '7': 34, 'months': 35, 'so': 36, 'becomes': 37, '799': 38, '*': 39, '=': 40, '5600': 41, 'approx': 42, 'syllabus': 43, 'we': 44, 'will': 45, 'be': 46, 'covering': 47, 'following': 48, 'modules': 49, ':': 50, 'python': 51, 'fundamentals': 52, 'libraries': 53, 'analysis': 54, 'sql': 55, 'maths': 56, 'machine': 57, 'learning': 58, 'ml': 59, 'algorithms': 60, 'practical': 61, 'mlops': 62, 'case': 63, 'studies': 64, 'can': 65, 'check': 66, 'detailed': 67, 'here': 68, '-': 69, 'https': 70, '//learnwith.campusx.in/courses/campusx-data-science-mentorship-program-637339afe4b0615a1bbed

In [59]:
# Extract sentences from data
inputSentences = document.split('\n') # Split document based on new line

In [60]:
# make a function to convert word into indices
def text_to_indices(sentence, vocab):
  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])
  return numerical_sentence

In [61]:
input_numerical_sentences = []
for sentence in inputSentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [62]:
input_numerical_sentences[0]

[1, 2, 3]

## Creating Training Sequences

In [63]:
training_sequences = [] # list to hold training sequence
for sentence in input_numerical_sentences:
  for i in range(1, len(sentence)):
    training_sequences.append(sentence[:i+1])

In [64]:
training_sequences

[[1, 2],
 [1, 2, 3],
 [4, 5],
 [4, 5, 2],
 [4, 5, 2, 6],
 [4, 5, 2, 6, 7],
 [4, 5, 2, 6, 7, 8],
 [4, 5, 2, 6, 7, 8, 9],
 [4, 5, 2, 6, 7, 8, 9, 10],
 [4, 5, 2, 6, 7, 8, 9, 10, 11],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15],
 [2, 6],
 [2, 6, 16],
 [2, 6, 16, 17],
 [2, 6, 16, 17, 18],
 [2, 6, 16, 17, 18, 19],
 [2, 6, 16, 17, 18, 19, 20],
 [2, 6, 16, 17, 18, 19, 20, 21],
 [2, 6, 16, 17, 18, 19, 20, 21, 22],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18

In [65]:
# List of total sequnces
len(training_sequences)

942

* We will use LSTM and the data will be processed into batches, we need to make padding to convert each input sequence of the same length.

In [66]:
# Get length of maximum training sequence
len_list = [len(sequence) for sequence in training_sequences]
max_len = max(len_list)

In [67]:
print(f"Maximum input sequence length is {max_len}")

Maximum input sequence length is 62


## Padding

In [68]:
from collections.abc import Sequence
# Perform padding to convert all sequnces into of same length
# we will add 0 in the start of sequences
padded_training_sequence = [[0] * (max_len - len(sequence)) + sequence for sequence in training_sequences]

In [69]:
len(padded_training_sequence[1])

62

In [70]:
# Convert the training sequence into tensors
padded_training_sequence = torch.tensor(padded_training_sequence, dtype = torch.long)

In [71]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287],
        [  0,   0,   0,  ..., 286, 287, 288]])

In [72]:
# Get the shape
padded_training_sequence.shape

torch.Size([942, 62])

In [73]:
# Split labels from sequence (label will be the last element in each tensor)
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:, -1]

# Dataset and DataLoader Classes

In [74]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [75]:
# Create dataset objects from CustomDataset class
dataset = CustomDataset(X, y)

In [76]:
# Create DataLoader object
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True, pin_memory=True)

In [77]:
for data in dataloader:
  print(data)
  break

[tensor([[  0,   0,   0,  ...,  17, 105, 240],
        [  0,   0,   0,  ...,  76,   5,  24],
        [  0,   0,   0,  ..., 111,  27,   2],
        ...,
        [  0,   0,   0,  ..., 255,  80, 116],
        [  0,   0,   0,  ...,  30,  36,  85],
        [  0,   0,   0,  ..., 268,  30,  36]]), tensor([233, 265,   6,  27,   2,  86,  30, 154, 241,  19,  85,  74,  93,  46,
          5, 159,  35,  45,   5,  24, 161,  22,   2, 165, 192,   7, 153, 213,
         27, 256,  22,  44])]


# LSTM Model

In [78]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 250, batch_first=True)
    self.fc = nn.Linear(250, vocab_size)
  def forward(self, x):
    embedded = self.embedding(x)
    int_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

## Model Training

In [79]:
model = LSTMModel(len(vocab))  # Create a model object
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {device}")
model.to(device)


Running on: cuda


LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=289, bias=True)
)

In [83]:
# Define Hyperparameters
epochs = 70
learning_rate = 0.001

In [84]:
# Define Loss
criterion = nn.CrossEntropyLoss()
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## Training Loop

In [85]:
# training loop
for epoch in range(epochs):
  total_loss = 0
  for batch_x, batch_y in dataloader:
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    # Zero out previous grads
    optimizer.zero_grad()

    # Forward pass
    output = model(batch_x)

    # Calculate Loss
    loss = criterion(output, batch_y)

    # Calculate gradients
    loss.backward()

    # Update parameters
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 2.8657
Epoch: 2, Loss: 2.8600
Epoch: 3, Loss: 2.7679
Epoch: 4, Loss: 2.6814
Epoch: 5, Loss: 2.6453
Epoch: 6, Loss: 2.6824
Epoch: 7, Loss: 2.7152
Epoch: 8, Loss: 2.6415
Epoch: 9, Loss: 2.6864
Epoch: 10, Loss: 2.6018
Epoch: 11, Loss: 2.6038
Epoch: 12, Loss: 2.7355
Epoch: 13, Loss: 2.6834
Epoch: 14, Loss: 2.7464
Epoch: 15, Loss: 2.5895
Epoch: 16, Loss: 2.7070
Epoch: 17, Loss: 2.6194
Epoch: 18, Loss: 2.6213
Epoch: 19, Loss: 2.7132
Epoch: 20, Loss: 2.5841
Epoch: 21, Loss: 2.5876
Epoch: 22, Loss: 2.7024
Epoch: 23, Loss: 2.6790
Epoch: 24, Loss: 2.6390
Epoch: 25, Loss: 2.7338
Epoch: 26, Loss: 2.5985
Epoch: 27, Loss: 2.6302
Epoch: 28, Loss: 2.6428
Epoch: 29, Loss: 2.6117
Epoch: 30, Loss: 2.6776
Epoch: 31, Loss: 2.5977
Epoch: 32, Loss: 2.7769
Epoch: 33, Loss: 2.6344
Epoch: 34, Loss: 2.6495
Epoch: 35, Loss: 2.5854
Epoch: 36, Loss: 2.5778
Epoch: 37, Loss: 2.6905
Epoch: 38, Loss: 2.5914
Epoch: 39, Loss: 2.6468
Epoch: 40, Loss: 2.6652
Epoch: 41, Loss: 2.6599
Epoch: 42, Loss: 2.6426
E

# Model Evaluation

In [86]:
# Set model to evaluation mode
model.eval()

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=289, bias=True)
)

In [89]:
# Define a prediction function
def prediction(model, vocab, text):
  # Tokenize text
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # Padding
  padded_text = torch.tensor([0] * (max_len - len(numerical_text)) + numerical_text, dtype = torch.long).unsqueeze(0)
  padded_text = padded_text.to(device)
  # Get output from model
  output = model(padded_text)

  # Get prediction index
  value, index = torch.max(output, dim = 1)

  # Merge with text
  return text +  " " + list(vocab.keys())[index]

In [90]:
num_tokens = 10 # max token generation
input_text = "pay the entire"
for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text


pay the entire fee
pay the entire fee of
pay the entire fee of rs
pay the entire fee of rs 5600
pay the entire fee of rs 5600 all
pay the entire fee of rs 5600 all at
pay the entire fee of rs 5600 all at once
pay the entire fee of rs 5600 all at once ?
pay the entire fee of rs 5600 all at once ? usp=sharing
pay the entire fee of rs 5600 all at once ? usp=sharing .


## Conclusion

This notebook demonstrates that even a simple LSTM model trained on a small corpus can generate the next word in a sequence with some degree of accuracy. The output shows that the model is able to predict plausible next words based on the input text, although the predictions become less coherent as the generated sequence lengthens.

To improve the model's performance and generate more accurate and contextually relevant next words, the following steps can be taken:

- **Use a larger corpus:** Training the model on a significantly larger dataset will expose it to a wider range of vocabulary and sentence structures, leading to better generalization and more diverse predictions.
- **Increase model complexity:** Experimenting with more complex LSTM architectures, such as adding more layers or increasing the hidden state size, can enhance the model's ability to capture intricate patterns in the data.
- **Incorporate advanced techniques:** Exploring techniques like attention mechanisms, transformers, or using pre-trained language models can further boost the model's predictive capabilities.

By implementing these improvements, we can build a more robust and sophisticated next-word prediction model.